In [9]:
from requests import get
import pandas as pd
from API_Keys import quandl

In [10]:
url = f"https://www.quandl.com/api/v3/datasets/CHRIS/CME_MGC1.json?api_key={quandl}"

In [15]:
pull = get(url)

In [17]:
pull = pull.json()
pull

{'dataset': {'id': 11061194,
  'dataset_code': 'CME_MGC1',
  'database_code': 'CHRIS',
  'name': 'E-micro Gold Futures, Continuous Contract #1 (MGC1) (Front Month)',
  'description': 'Historical Futures Prices: E-micro Gold Futures, Continuous Contract #1. Non-adjusted price based on spot-month continuous contract calculations. Raw data from CME. ',
  'refreshed_at': '2019-07-23T02:26:00.634Z',
  'newest_available_date': '2019-07-22',
  'oldest_available_date': '2014-03-05',
  'column_names': ['Date',
   'Open',
   'High',
   'Low',
   'Last',
   'Change',
   'Settle',
   'Volume',
   'Previous Day Open Interest'],
  'frequency': 'daily',
  'type': 'Time Series',
  'premium': False,
  'limit': None,
  'transform': None,
  'column_index': None,
  'start_date': '2014-03-05',
  'end_date': '2019-07-22',
  'data': [['2019-07-22',
    1428.0,
    1431.3,
    1422.9,
    1425.4,
    0.2,
    1426.9,
    17359.0,
    8353.0],
   ['2019-07-19',
    1448.4,
    1454.5,
    1421.1,
    1426.6,
 

In [23]:
df = pd.DataFrame(pull["dataset"]["data"],columns=pull["dataset"]["column_names"])

In [24]:
df.head()

,Date,Open,High,Low,Last,Change,Settle,Volume,Previous Day Open Interest
0,2019-07-22,1428.0,1431.3,1422.9,1425.4,0.2,1426.9,17359.0,8353.0
1,2019-07-19,1448.4,1454.5,1421.1,1426.6,1.4,1426.7,38071.0,7504.0
2,2019-07-18,1431.6,1450.0,1415.7,1447.5,4.8,1428.1,35386.0,7697.0
3,2019-07-17,1407.8,1428.3,1400.7,1428.1,12.1,1423.3,24880.0,8420.0
4,2019-07-16,1416.3,1420.0,1402.1,1407.7,2.3,1411.2,23054.0,8265.0
